In [ ]:
# import sys
# from random import random
# from operator import add

# from pyspark.sql import SparkSession


# if __name__ == "__main__":
#     """
#         Usage: pi [partitions]
#     """
#     spark = SparkSession\
#         .builder\
#         .appName("PythonPi")\
#         .getOrCreate()

#     partitions = int(sys.argv[1]) if len(sys.argv) > 1 else 2
#     n = 100000 * partitions

#     def f(_):
#         x = random() * 2 - 1
#         y = random() * 2 - 1
#         return 1 if x ** 2 + y ** 2 <= 1 else 0

#     count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
#     print("Pi is roughly %f" % (4.0 * count / n))

#     spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

print("Started Processing Labels!")
if __name__ == "__main__":
    spark = SparkSession.builder.appName("labelGen").getOrCreate()
    df = spark.read.parquet("s3://ds102-coolteam-scratch/preprocessing.parquet")
    def currentLoanHelper(val):
        if int(val) >= 3:
            return 1
        else:
            return 0

    def zeroBalanceHelper(val):
        target = [3,6,9]
        if val in target:
            return 1
        else:
            return 0
    def defaultHelper(val):
        if val == 0:
            return 0
        else:
            return 1

    currentLoanHelperUDF = F.udf(currentLoanHelper, IntegerType())
    zeroBalanceHelperUDF = F.udf(zeroBalanceHelper, IntegerType())
    defaultHelperUDF = F.udf(defaultHelper, IntegerType())

    df = df.withColumn('CLH', currentLoanHelperUDF('CurrentLoanDelinquencyStatus'))
    df = df.withColumn('ZBH', zeroBalanceHelperUDF('ZeroBalanceCode'))
    
    df = df.withColumn('Default', F.expr("CLH + ZBH"))
    df = df.withColumn('Default', defaultHelperUDF('Default'))
    df = df.drop("CLH","ZBH","Current LoanDelinquencyStatus", "ZeroBalanceCode")
    df.write.parquet("s3://ds102-coolteam-scratch/labels.parquet")
    print("Finished Processing Labels!")
    spark.stop()